In [11]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import json
import io
import csv
import shutil
from tqdm import tqdm
from collections import defaultdict

In [2]:
#进行数据清洗的测试，从训练集中读取数据
data = pd.read_csv('train_usual.csv',header=None)
#加入列名
data.columns=["label","text"]

In [3]:
data.head()

,label,text
0,1,气死 姐姐 了 ， 快二是 阵亡 了 吗 ， 尼玛 ， 一个半 小时 过去 了 也 没 上车
1,2,妞妞 啊 ， 今天 又 承办 了 一个 发文 登记 文号 是 126 ~ 嘻 ~ 么 么 哒...
2,6,这里 还 值得注意 另 一个 事实 ， 就是 张鞠存 原有 一个 东溪 草堂 为 其 读书处 。
3,6,这 在 前 华约 国家 ( 尤其 是 东德 ) 使用 R - 73 的 首次 联合演习 期间...
4,5,TinyThief 上 wii 了 ？ ！


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27766 entries, 0 to 27765
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   27766 non-null  int64 
 1   text    27766 non-null  object
dtypes: int64(1), object(1)
memory usage: 434.0+ KB


In [5]:
data['label'].value_counts()

1    8343
6    5749
2    5378
3    4990
5    2086
4    1220
Name: label, dtype: int64

In [6]:
data.describe()

,label
count,27766.000000
mean,3.020709
std,1.898212
min,1.000000
25%,1.000000
50%,3.000000
75%,5.000000
max,6.000000


In [7]:
# 查看每一列是否有缺失：
data.isna().any(axis=0)
# 查看每一行是否有缺失：
data.isna().any(axis=1)
# 查看所有数据中是否有缺少最快的，没有输出False，反之为True：
data.isna().values.any()
#data[data.isna().values==True]

False

In [8]:
data['text'].map(len).describe()
#计算文本的长度，其中mean为均值

count    27766.000000
mean        69.081142
std         48.949356
min         13.000000
25%         35.000000
50%         52.000000
75%         84.000000
max        371.000000
Name: text, dtype: float64

In [9]:
#data['label'].value_counts(normalize=True).plot(kind='bar');
data['label'].value_counts()

1    8343
6    5749
2    5378
3    4990
5    2086
4    1220
Name: label, dtype: int64

In [10]:
data = pd.read_csv('test_usual.csv',header=None)
data.columns=["label","text"]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   5000 non-null   int64 
 1   text    5000 non-null   object
dtypes: int64(1), object(1)
memory usage: 78.2+ KB


In [11]:
data = pd.read_csv('dev_usual.csv',header=None)
data.columns=["label","text"]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   2000 non-null   int64 
 1   text    2000 non-null   object
dtypes: int64(1), object(1)
memory usage: 31.4+ KB


In [12]:
#定义数据清洗函数
def clean_data(old_path,save_path,save_name):
    with open(old_path, 'r', encoding='utf8') as f:
        data = defaultdict(list)
        texts = f.readlines()
        for line in tqdm(texts, desc=old_path):
            label,text= line.strip().split(',',1)
            emoji = json.load(open('C:/Users/23106/Desktop/weibopy/emoji.json', 'r', encoding='utf8'))
            for emoji, emoji_text in emoji.items():
                text = text.replace(emoji,emoji_text )
            new_text=re.sub('[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~\s]+', "", text)
            data['text'].append(new_text)
            
            data['label'].append(label)
            #将label与text交换位置
        df = pd.DataFrame(data)
        df.to_csv(os.path.join(save_path,save_name), index=False,encoding='utf8', header=False, sep='\t')
    return

In [14]:
clean_data('C:/Users/23106/Desktop/weibopy/dev_usual.csv','C:/Users/23106/Desktop/weibopy/new/','valid.csv')

C:/Users/23106/Desktop/weibopy/dev_usual.csv: 100%|██████████| 2000/2000 [00:01<00:00, 1308.94it/s]


In [14]:
#从经过数据清洗后生成的文件中读取数据
data = pd.read_csv('C:/Users/23106/Desktop/weibopy/new/train.csv',sep='\t',header=None)
#加入列名
data.columns=["text","label"]

In [15]:
data.head(15)

,text,label
0,气死姐姐了快二是阵亡了吗尼玛一个半小时过去了也没上车,1
1,妞妞啊今天又承办了一个发文登记文号是嘻么么哒晚安哟,2
2,这里还值得注意另一个事实就是张鞠存原有一个东溪草堂为其读书处,6
3,这在前华约国家尤其是东德使用的首次联合演习期间被一些北约组织的飞行员所证实,6
4,上了,5
5,每天都以紧张的心情工作真的是太累太不放松了想要爆发一下,1
6,语文军数学军英语军物理军政治军历史军生物军地理军八科联军侵犯我班我班战败被迫签订家长会条约赔...,1
7,我不是一个优秀的演员不能微笑着旁观你们幸福,3
8,当你变优秀时你想要的都会来找你,2
9,累了一天会宿舍听下我搞基新歌在看看我段宜恩美图心都被治愈了,2


In [16]:
#将label中的数字进行转化，便于查看
data.loc[data['label']==1, 'label'] = '愤怒'
data.loc[data['label']==2, 'label'] = '积极'
data.loc[data['label']==3, 'label'] = '悲伤'
data.loc[data['label']==4, 'label'] = '恐惧'
data.loc[data['label']==5, 'label'] = '惊奇'
data.loc[data['label']==6, 'label'] = '无情绪'

In [17]:
data.head(14)

,text,label
0,气死姐姐了快二是阵亡了吗尼玛一个半小时过去了也没上车,愤怒
1,妞妞啊今天又承办了一个发文登记文号是嘻么么哒晚安哟,积极
2,这里还值得注意另一个事实就是张鞠存原有一个东溪草堂为其读书处,无情绪
3,这在前华约国家尤其是东德使用的首次联合演习期间被一些北约组织的飞行员所证实,无情绪
4,上了,惊奇
5,每天都以紧张的心情工作真的是太累太不放松了想要爆发一下,愤怒
6,语文军数学军英语军物理军政治军历史军生物军地理军八科联军侵犯我班我班战败被迫签订家长会条约赔...,愤怒
7,我不是一个优秀的演员不能微笑着旁观你们幸福,悲伤
8,当你变优秀时你想要的都会来找你,积极
9,累了一天会宿舍听下我搞基新歌在看看我段宜恩美图心都被治愈了,积极


In [18]:
data = data[['text', 'label']]
data.to_csv('C:/Users/23106/Desktop/weibopy/new/train.csv', index=False, sep="\t")

In [19]:
#从经过数据清洗后生成的文件中读取数据
data = pd.read_csv('C:/Users/23106/Desktop/weibopy/new/test.csv',sep='\t',header=None)
#加入列名
data.columns=["text","label"]
#将label中的数字进行转化，便于查看
data.loc[data['label']==1, 'label'] = '愤怒'
data.loc[data['label']==2, 'label'] = '积极'
data.loc[data['label']==3, 'label'] = '悲伤'
data.loc[data['label']==4, 'label'] = '恐惧'
data.loc[data['label']==5, 'label'] = '惊奇'
data.loc[data['label']==6, 'label'] = '无情绪'
data = data[['text', 'label']]
data.to_csv('C:/Users/23106/Desktop/weibopy/new/test.csv', index=False, sep="\t")

In [20]:
#从经过数据清洗后生成的文件中读取数据
data = pd.read_csv('C:/Users/23106/Desktop/weibopy/new/valid.csv',sep='\t',header=None)
#加入列名
data.columns=["text","label"]
#将label中的数字进行转化，便于查看
data.loc[data['label']==1, 'label'] = '愤怒'
data.loc[data['label']==2, 'label'] = '积极'
data.loc[data['label']==3, 'label'] = '悲伤'
data.loc[data['label']==4, 'label'] = '恐惧'
data.loc[data['label']==5, 'label'] = '惊奇'
data.loc[data['label']==6, 'label'] = '无情绪'
data = data[['text', 'label']]
data.to_csv('C:/Users/23106/Desktop/weibopy/new/valid.csv', index=False, sep="\t")

In [21]:
#读取数据
train_data = pd.read_csv('C:/Users/23106/Desktop/weibopy/new/train.csv',sep='\t',header=None)
train_data.columns=["label","text"]
test_data = pd.read_csv('C:/Users/23106/Desktop/weibopy/new/test.csv',sep='\t',header=None)
test_data.columns=["label","text"]

In [22]:
test_data.head()

,label,text
0,text,label
1,宿舍要民汉合宿了为毛都大三了还要折腾我,愤怒
2,早上的我竟然也变成了一个无理取闹的人,悲伤
3,多年来周天先生率智多星的律师策划师团队走出巴蜀挺进海西在厦门特区的发祥地安营扎寨,无情绪
4,听故事的时候你总喜欢眼巴巴的问后来呢可是当后来自己成为讲故事的人才发现后来故事和话语就在嘴边...,悲伤


In [23]:
train_data.head()

,label,text
0,text,label
1,气死姐姐了快二是阵亡了吗尼玛一个半小时过去了也没上车,愤怒
2,妞妞啊今天又承办了一个发文登记文号是嘻么么哒晚安哟,积极
3,这里还值得注意另一个事实就是张鞠存原有一个东溪草堂为其读书处,无情绪
4,这在前华约国家尤其是东德使用的首次联合演习期间被一些北约组织的飞行员所证实,无情绪


In [24]:
#查看测试集信息
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5001 entries, 0 to 5000
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   5000 non-null   object
 1   text    5001 non-null   object
dtypes: object(2)
memory usage: 78.3+ KB


In [25]:
#构建情感分析模型
import paddle

In [26]:
import paddlehub as hub

In [27]:
#检测环境是否正常
paddle.utils.run_check()

Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 GPU.
PaddlePaddle works well on 1 GPUs.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


In [28]:
label_list=list(data.label.unique())
print(label_list)

['愤怒', '积极', '悲伤', '无情绪', '恐惧', '惊奇']


In [29]:
label_map = { 
    idx: label_text for idx, label_text in enumerate(label_list)
}
print(label_map)

{0: '愤怒', 1: '积极', 2: '悲伤', 3: '无情绪', 4: '恐惧', 5: '惊奇'}


In [30]:
# 此处选择ernie_tiny预训练模型并设置微调任务为6分类任务
model = hub.Module(name="ernie_tiny", task='seq-cls', num_classes=6, label_map=label_map)

[2022-05-10 15:56:12,373] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\ernie_tiny.pdparams


In [31]:
from paddlehub.datasets.base_nlp_dataset import InputExample, TextClassificationDataset

In [32]:
import os, io, csv

In [33]:
print(label_list)

['愤怒', '积极', '悲伤', '无情绪', '恐惧', '惊奇']


In [34]:
DIR="C:/Users/23106/Desktop/weibopy/new/"

In [35]:
class TEXTALS(TextClassificationDataset):
    def __init__(self, tokenizer, mode='train', max_seq_len=128):
        if mode == 'train':
            data_file = 'train.csv'  # 训练集
        elif mode == 'test':
            data_file = 'test.csv'   # 测试集
        else:
            data_file = 'valid.csv'  # 验证集
        
        super(TEXTALS, self).__init__(
            base_path=DIR,
            data_file=data_file,
            tokenizer=tokenizer,
            max_seq_len=max_seq_len,
            mode=mode,
            is_file_with_header=True,
            label_list=label_list
            )

    # 解析文本文件里的样本
    def _read_file(self, input_file, is_file_with_header: bool = False):
        if not os.path.exists(input_file):
            raise RuntimeError("The file {} is not found.".format(input_file))
        else:
            with io.open(input_file, "r", encoding="UTF-8") as f:
                reader = csv.reader(f, delimiter="\t")
                examples = []
                seq_id = 0
                header = next(reader) if is_file_with_header else None
                for line in reader:
                    try:
                        example = InputExample(guid=seq_id, text_a=line[0], label=line[1])
                        seq_id += 1
                        examples.append(example)
                    except:
                        continue
                return examples
                
train_dataset = TEXTALS(model.get_tokenizer(), mode='train', max_seq_len=128)
test_dataset = TEXTALS(model.get_tokenizer(), mode='test', max_seq_len=128)
valid_dataset = TEXTALS(model.get_tokenizer(), mode='valid', max_seq_len=128)

# 查看训练集前3条
for e in train_dataset.examples[:3]:
    print(e)
# 查看验证集前3条
for e in test_dataset.examples[:3]:
    print(e)
# 查看测试集前3条
for e in valid_dataset.examples[:3]:
    print(e)

[2022-05-10 15:56:14,271] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\vocab.txt
[2022-05-10 15:56:14,272] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\spm_cased_simp_sampled.model
[2022-05-10 15:56:14,273] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\dict.wordseg.pickle
[2022-05-10 15:56:22,779] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\vocab.txt
[2022-05-10 15:56:22,780] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\spm_cased_simp_sampled.model
[2022-05-10 15:56:22,780] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\dict.wordseg.pickle
[2022-05-10 15:56:26,706] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\vocab.txt
[2022-05-10 15:56:26,707] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\spm_cased_simp_sampled.model
[2022-05-10 15:56:26,709] [    INFO] - Already cached C:\Us

text=气死姐姐了快二是阵亡了吗尼玛一个半小时过去了也没上车	label=愤怒
text=妞妞啊今天又承办了一个发文登记文号是嘻么么哒晚安哟	label=积极
text=这里还值得注意另一个事实就是张鞠存原有一个东溪草堂为其读书处	label=无情绪
text=宿舍要民汉合宿了为毛都大三了还要折腾我	label=愤怒
text=早上的我竟然也变成了一个无理取闹的人	label=悲伤
text=多年来周天先生率智多星的律师策划师团队走出巴蜀挺进海西在厦门特区的发祥地安营扎寨	label=无情绪
text=所以注定我这辈子是做不了商人妈蛋	label=愤怒
text=无论是心情多么低沉的夜晚天光大亮后都是崭新的开始	label=积极
text=帽子怎么就变绿色幸好只是试一下	label=悲伤


In [36]:
#选择优化策略和运行配置
optimizer = paddle.optimizer.AdamW(learning_rate=4e-5, parameters=model.parameters())
trainer = hub.Trainer(model, optimizer, checkpoint_dir='./ckpt', use_gpu=True, use_vdl=True)

[2022-05-10 15:56:30,109] [ WARNING] - PaddleHub model checkpoint not found, start from scratch...


In [37]:
print(paddle.device.get_device())

gpu:0


In [38]:
trainer.train(train_dataset, epochs=6, batch_size=32, eval_dataset=valid_dataset, save_interval=1)

[2022-05-10 15:56:36,213] [   TRAIN] - Epoch=1/6, Step=10/868 loss=1.6585 acc=0.3531 lr=0.000040 step/sec=1.65 | ETA 00:52:44
[2022-05-10 15:56:42,211] [   TRAIN] - Epoch=1/6, Step=20/868 loss=1.3667 acc=0.5188 lr=0.000040 step/sec=1.67 | ETA 00:52:24
[2022-05-10 15:56:48,232] [   TRAIN] - Epoch=1/6, Step=30/868 loss=1.1766 acc=0.6219 lr=0.000040 step/sec=1.66 | ETA 00:52:21
[2022-05-10 15:56:54,259] [   TRAIN] - Epoch=1/6, Step=40/868 loss=1.1105 acc=0.6219 lr=0.000040 step/sec=1.66 | ETA 00:52:20
[2022-05-10 15:57:00,303] [   TRAIN] - Epoch=1/6, Step=50/868 loss=0.8493 acc=0.7188 lr=0.000040 step/sec=1.65 | ETA 00:52:22
[2022-05-10 15:57:06,330] [   TRAIN] - Epoch=1/6, Step=60/868 loss=0.8801 acc=0.6750 lr=0.000040 step/sec=1.66 | ETA 00:52:21
[2022-05-10 15:57:12,336] [   TRAIN] - Epoch=1/6, Step=70/868 loss=0.8899 acc=0.6875 lr=0.000040 step/sec=1.67 | ETA 00:52:19
[2022-05-10 15:57:18,310] [   TRAIN] - Epoch=1/6, Step=80/868 loss=0.9116 acc=0.6719 lr=0.000040 step/sec=1.67 | ETA 0

[2022-05-10 16:02:41,309] [   TRAIN] - Epoch=1/6, Step=600/868 loss=0.7366 acc=0.7375 lr=0.000040 step/sec=1.62 | ETA 00:53:41
[2022-05-10 16:02:47,442] [   TRAIN] - Epoch=1/6, Step=610/868 loss=0.5847 acc=0.7937 lr=0.000040 step/sec=1.63 | ETA 00:53:41
[2022-05-10 16:02:53,610] [   TRAIN] - Epoch=1/6, Step=620/868 loss=0.7235 acc=0.7312 lr=0.000040 step/sec=1.62 | ETA 00:53:41
[2022-05-10 16:02:59,863] [   TRAIN] - Epoch=1/6, Step=630/868 loss=0.7007 acc=0.7344 lr=0.000040 step/sec=1.60 | ETA 00:53:41
[2022-05-10 16:03:06,093] [   TRAIN] - Epoch=1/6, Step=640/868 loss=0.6611 acc=0.7625 lr=0.000040 step/sec=1.61 | ETA 00:53:42
[2022-05-10 16:03:12,249] [   TRAIN] - Epoch=1/6, Step=650/868 loss=0.7017 acc=0.7250 lr=0.000040 step/sec=1.62 | ETA 00:53:41
[2022-05-10 16:03:18,554] [   TRAIN] - Epoch=1/6, Step=660/868 loss=0.6738 acc=0.7688 lr=0.000040 step/sec=1.59 | ETA 00:53:42
[2022-05-10 16:03:25,063] [   TRAIN] - Epoch=1/6, Step=670/868 loss=0.7263 acc=0.7219 lr=0.000040 step/sec=1.54

[2022-05-10 16:08:55,760] [   TRAIN] - Epoch=2/6, Step=310/868 loss=0.5036 acc=0.8219 lr=0.000040 step/sec=1.62 | ETA 00:54:56
[2022-05-10 16:09:02,025] [   TRAIN] - Epoch=2/6, Step=320/868 loss=0.4717 acc=0.8281 lr=0.000040 step/sec=1.60 | ETA 00:54:56
[2022-05-10 16:09:08,204] [   TRAIN] - Epoch=2/6, Step=330/868 loss=0.4700 acc=0.8344 lr=0.000040 step/sec=1.62 | ETA 00:54:55
[2022-05-10 16:09:14,378] [   TRAIN] - Epoch=2/6, Step=340/868 loss=0.5841 acc=0.7844 lr=0.000040 step/sec=1.62 | ETA 00:54:54
[2022-05-10 16:09:20,567] [   TRAIN] - Epoch=2/6, Step=350/868 loss=0.5489 acc=0.7906 lr=0.000040 step/sec=1.62 | ETA 00:54:54
[2022-05-10 16:09:26,881] [   TRAIN] - Epoch=2/6, Step=360/868 loss=0.3856 acc=0.8781 lr=0.000040 step/sec=1.58 | ETA 00:54:54
[2022-05-10 16:09:33,133] [   TRAIN] - Epoch=2/6, Step=370/868 loss=0.5291 acc=0.7969 lr=0.000040 step/sec=1.60 | ETA 00:54:53
[2022-05-10 16:09:39,285] [   TRAIN] - Epoch=2/6, Step=380/868 loss=0.5058 acc=0.8031 lr=0.000040 step/sec=1.63

[2022-05-10 16:15:05,465] [   TRAIN] - Epoch=3/6, Step=20/868 loss=0.3033 acc=0.8938 lr=0.000040 step/sec=1.67 | ETA 00:55:07
[2022-05-10 16:15:11,463] [   TRAIN] - Epoch=3/6, Step=30/868 loss=0.3090 acc=0.9094 lr=0.000040 step/sec=1.67 | ETA 00:55:06
[2022-05-10 16:15:17,467] [   TRAIN] - Epoch=3/6, Step=40/868 loss=0.2730 acc=0.9125 lr=0.000040 step/sec=1.67 | ETA 00:55:05
[2022-05-10 16:15:23,466] [   TRAIN] - Epoch=3/6, Step=50/868 loss=0.3600 acc=0.8938 lr=0.000040 step/sec=1.67 | ETA 00:55:04
[2022-05-10 16:15:29,482] [   TRAIN] - Epoch=3/6, Step=60/868 loss=0.3309 acc=0.8781 lr=0.000040 step/sec=1.66 | ETA 00:55:03
[2022-05-10 16:15:35,645] [   TRAIN] - Epoch=3/6, Step=70/868 loss=0.2993 acc=0.8938 lr=0.000040 step/sec=1.62 | ETA 00:55:03
[2022-05-10 16:15:41,671] [   TRAIN] - Epoch=3/6, Step=80/868 loss=0.2912 acc=0.8781 lr=0.000040 step/sec=1.66 | ETA 00:55:02
[2022-05-10 16:15:47,666] [   TRAIN] - Epoch=3/6, Step=90/868 loss=0.3286 acc=0.8844 lr=0.000040 step/sec=1.67 | ETA 0

[2022-05-10 16:20:59,725] [   TRAIN] - Epoch=3/6, Step=610/868 loss=0.4173 acc=0.8531 lr=0.000040 step/sec=1.67 | ETA 00:54:22
[2022-05-10 16:21:05,727] [   TRAIN] - Epoch=3/6, Step=620/868 loss=0.3158 acc=0.8875 lr=0.000040 step/sec=1.67 | ETA 00:54:21
[2022-05-10 16:21:11,724] [   TRAIN] - Epoch=3/6, Step=630/868 loss=0.4295 acc=0.8313 lr=0.000040 step/sec=1.67 | ETA 00:54:21
[2022-05-10 16:21:17,723] [   TRAIN] - Epoch=3/6, Step=640/868 loss=0.3607 acc=0.8812 lr=0.000040 step/sec=1.67 | ETA 00:54:20
[2022-05-10 16:21:23,720] [   TRAIN] - Epoch=3/6, Step=650/868 loss=0.2946 acc=0.8844 lr=0.000040 step/sec=1.67 | ETA 00:54:20
[2022-05-10 16:21:29,721] [   TRAIN] - Epoch=3/6, Step=660/868 loss=0.3692 acc=0.8656 lr=0.000040 step/sec=1.67 | ETA 00:54:19
[2022-05-10 16:21:35,726] [   TRAIN] - Epoch=3/6, Step=670/868 loss=0.3415 acc=0.8812 lr=0.000040 step/sec=1.67 | ETA 00:54:18
[2022-05-10 16:21:41,725] [   TRAIN] - Epoch=3/6, Step=680/868 loss=0.3515 acc=0.8719 lr=0.000040 step/sec=1.67

[2022-05-10 16:27:07,613] [   TRAIN] - Epoch=4/6, Step=330/868 loss=0.2148 acc=0.9313 lr=0.000040 step/sec=1.66 | ETA 00:54:21
[2022-05-10 16:27:13,646] [   TRAIN] - Epoch=4/6, Step=340/868 loss=0.1989 acc=0.9375 lr=0.000040 step/sec=1.66 | ETA 00:54:21
[2022-05-10 16:27:19,667] [   TRAIN] - Epoch=4/6, Step=350/868 loss=0.2010 acc=0.9187 lr=0.000040 step/sec=1.66 | ETA 00:54:20
[2022-05-10 16:27:25,697] [   TRAIN] - Epoch=4/6, Step=360/868 loss=0.2569 acc=0.9250 lr=0.000040 step/sec=1.66 | ETA 00:54:20
[2022-05-10 16:27:31,723] [   TRAIN] - Epoch=4/6, Step=370/868 loss=0.1931 acc=0.9250 lr=0.000040 step/sec=1.66 | ETA 00:54:19
[2022-05-10 16:27:37,731] [   TRAIN] - Epoch=4/6, Step=380/868 loss=0.1633 acc=0.9437 lr=0.000040 step/sec=1.66 | ETA 00:54:19
[2022-05-10 16:27:43,731] [   TRAIN] - Epoch=4/6, Step=390/868 loss=0.1640 acc=0.9406 lr=0.000040 step/sec=1.67 | ETA 00:54:19
[2022-05-10 16:27:49,730] [   TRAIN] - Epoch=4/6, Step=400/868 loss=0.1882 acc=0.9469 lr=0.000040 step/sec=1.67

[2022-05-10 16:33:15,800] [   TRAIN] - Epoch=5/6, Step=50/868 loss=0.1035 acc=0.9594 lr=0.000040 step/sec=1.66 | ETA 00:54:21
[2022-05-10 16:33:21,830] [   TRAIN] - Epoch=5/6, Step=60/868 loss=0.0810 acc=0.9719 lr=0.000040 step/sec=1.66 | ETA 00:54:21
[2022-05-10 16:33:27,848] [   TRAIN] - Epoch=5/6, Step=70/868 loss=0.0727 acc=0.9844 lr=0.000040 step/sec=1.66 | ETA 00:54:20
[2022-05-10 16:33:33,871] [   TRAIN] - Epoch=5/6, Step=80/868 loss=0.1170 acc=0.9625 lr=0.000040 step/sec=1.66 | ETA 00:54:20
[2022-05-10 16:33:39,895] [   TRAIN] - Epoch=5/6, Step=90/868 loss=0.1638 acc=0.9375 lr=0.000040 step/sec=1.66 | ETA 00:54:20
[2022-05-10 16:33:45,913] [   TRAIN] - Epoch=5/6, Step=100/868 loss=0.1179 acc=0.9531 lr=0.000040 step/sec=1.66 | ETA 00:54:19
[2022-05-10 16:33:51,946] [   TRAIN] - Epoch=5/6, Step=110/868 loss=0.0933 acc=0.9750 lr=0.000040 step/sec=1.66 | ETA 00:54:19
[2022-05-10 16:33:57,965] [   TRAIN] - Epoch=5/6, Step=120/868 loss=0.1137 acc=0.9594 lr=0.000040 step/sec=1.66 | ET

[2022-05-10 16:39:11,503] [   TRAIN] - Epoch=5/6, Step=640/868 loss=0.1070 acc=0.9563 lr=0.000040 step/sec=1.66 | ETA 00:54:04
[2022-05-10 16:39:17,520] [   TRAIN] - Epoch=5/6, Step=650/868 loss=0.1810 acc=0.9344 lr=0.000040 step/sec=1.66 | ETA 00:54:03
[2022-05-10 16:39:23,540] [   TRAIN] - Epoch=5/6, Step=660/868 loss=0.1085 acc=0.9688 lr=0.000040 step/sec=1.66 | ETA 00:54:03
[2022-05-10 16:39:29,557] [   TRAIN] - Epoch=5/6, Step=670/868 loss=0.1506 acc=0.9437 lr=0.000040 step/sec=1.66 | ETA 00:54:03
[2022-05-10 16:39:35,585] [   TRAIN] - Epoch=5/6, Step=680/868 loss=0.1843 acc=0.9313 lr=0.000040 step/sec=1.66 | ETA 00:54:03
[2022-05-10 16:39:41,610] [   TRAIN] - Epoch=5/6, Step=690/868 loss=0.1624 acc=0.9281 lr=0.000040 step/sec=1.66 | ETA 00:54:02
[2022-05-10 16:39:47,637] [   TRAIN] - Epoch=5/6, Step=700/868 loss=0.2495 acc=0.9125 lr=0.000040 step/sec=1.66 | ETA 00:54:02
[2022-05-10 16:39:53,659] [   TRAIN] - Epoch=5/6, Step=710/868 loss=0.1141 acc=0.9563 lr=0.000040 step/sec=1.66

[2022-05-10 16:45:25,899] [   TRAIN] - Epoch=6/6, Step=360/868 loss=0.0593 acc=0.9812 lr=0.000040 step/sec=1.64 | ETA 00:54:13
[2022-05-10 16:45:32,017] [   TRAIN] - Epoch=6/6, Step=370/868 loss=0.0727 acc=0.9719 lr=0.000040 step/sec=1.63 | ETA 00:54:12
[2022-05-10 16:45:38,140] [   TRAIN] - Epoch=6/6, Step=380/868 loss=0.1115 acc=0.9625 lr=0.000040 step/sec=1.63 | ETA 00:54:12
[2022-05-10 16:45:44,259] [   TRAIN] - Epoch=6/6, Step=390/868 loss=0.1153 acc=0.9563 lr=0.000040 step/sec=1.63 | ETA 00:54:12
[2022-05-10 16:45:50,378] [   TRAIN] - Epoch=6/6, Step=400/868 loss=0.0737 acc=0.9781 lr=0.000040 step/sec=1.63 | ETA 00:54:12
[2022-05-10 16:45:56,495] [   TRAIN] - Epoch=6/6, Step=410/868 loss=0.1255 acc=0.9625 lr=0.000040 step/sec=1.63 | ETA 00:54:12
[2022-05-10 16:46:02,618] [   TRAIN] - Epoch=6/6, Step=420/868 loss=0.0930 acc=0.9688 lr=0.000040 step/sec=1.63 | ETA 00:54:12
[2022-05-10 16:46:08,743] [   TRAIN] - Epoch=6/6, Step=430/868 loss=0.1316 acc=0.9656 lr=0.000040 step/sec=1.63

In [39]:
# 在测试集上评估当前训练模型
result = trainer.evaluate(test_dataset, batch_size=32)

[2022-05-10 16:51:26,142] [    EVAL] - [Evaluation result] avg_acc=0.7490m


In [40]:
data = [
    # 悲伤
    ["你也不用说对不起,只是若相惜"],
    # 积极
    ["幸福其实很简单"],
    # 恐惧
    ["恐惧感啊。生病"],
    # 积极
    ["待你长发及腰,我们一起耕耘时光。我愿等待"]
]
label_list=['愤怒', '积极', '悲伤', '无情绪', '恐惧', '惊奇']
label_map = {
    idx: label_text for idx, label_text in enumerate(label_list)
}
model = hub.Module(
    name='ernie_tiny',
    task='seq-cls',
    num_classes=6,
    load_checkpoint='./ckpt/best_model/model.pdparams',
    label_map=label_map)
results = model.predict(data, max_seq_len=128, batch_size=32, use_gpu=True)
for idx, text in enumerate(data):
    print('Data: {} \t Lable: {}'.format(text[0], results[idx]))

[2022-05-10 16:51:26,190] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\ernie_tiny.pdparams
[2022-05-10 16:51:29,198] [    INFO] - Loaded parameters from C:\Users\23106\Desktop\weibopy\ckpt\best_model\model.pdparams
[2022-05-10 16:51:29,209] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\vocab.txt
[2022-05-10 16:51:29,216] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\spm_cased_simp_sampled.model
[2022-05-10 16:51:29,217] [    INFO] - Already cached C:\Users\23106\.paddlenlp\models\ernie-tiny\dict.wordseg.pickle


Data: 你也不用说对不起,只是若相惜 	 Lable: 悲伤
Data: 幸福其实很简单 	 Lable: 积极
Data: 恐惧感啊。生病 	 Lable: 恐惧
Data: 待你长发及腰,我们一起耕耘时光。我愿等待 	 Lable: 积极


In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import json
import re
def fetchUrl(pid, uid, max_id):
    # url
    url = "https://weibo.com/ajax/statuses/buildComments"
    # 请求头
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36",
    }
    # 参数
    params = {
        "flow": 0,
        "is_reload": 1,
        "id": pid,
        "is_show_bulletin": 3,
        "is_mix": 0,
        "max_id": max_id,
        "count": 20,
        "uid": uid,
    }

    r = requests.get(url, headers=headers, params=params)
    return r.json()


def parseJson(jsonObj):

    data = jsonObj["data"]
    max_id = jsonObj["max_id"]
    commentData = []
    for item in data:
        # 评论id
        comment_Id = item["id"]
        # 评论内容
        content = BeautifulSoup(item["text"], "html.parser").text
        # 评论时间
        created_at = item["created_at"]
        # 点赞数
        like_counts = item["like_counts"]
        # 评论数
        total_number = item["total_number"]

        # 评论者 id，name，city
        user = item["user"]
        userID = user["id"]
        userName = user["name"]
        userCity = user["location"]
        content=re.sub('[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~\s]+', "", content)
        emoji = json.load(open('C:/Users/23106/Desktop/weibopy/emoji.json', 'r', encoding='utf8'))
        for emoji, emoji_text in emoji.items():
            content = content.replace(emoji,emoji_text )
        dataItem = [content]
        print(dataItem)
        commentData.append(dataItem)

    return commentData, max_id
def save_data(data, path, filename):
    
    if not os.path.exists(path):
        os.makedirs(path)

    dataframe = pd.DataFrame(data)
    dataframe.to_csv(path + filename, mode='a', index=False, sep=',', header=False )
if __name__ == "__main__":
    
    pid = 4727457598933626      # 微博id，固定
    uid = 2140522467            # 用户id，固定
    max_id = 0
    path = "C:/Users/23106/Desktop/weibopy/new/"           # 保存的路径
    filename = "comments.csv"   # 保存的文件名
    
    #csvHeader = [["评论id", "发布时间", "用户id", "用户昵称", "用户城市", "点赞数", "回复数", "评论内容"]]
    #save_data(csvHeader, path, filename)

    while(True):
        html = fetchUrl(pid, uid, max_id)
        comments, max_id = parseJson(html)
        save_data(comments, path, filename)
        # max_id 为 0 时，表示爬取结束
        if max_id == 0:
            break;

['强化班一直到考研结束吗']
['老师我买了咱们的课可是视频强化班只有马原没有别的怎们办是没更新吗考研政治徐涛']
['强化班有多少节一共多长时间能看完']
['都了怎么评论区还是一群老熟人']
['终于学生都要跑走了再不营业就剩我们这些老家伙了']
['今年的核心考案又有一些小改动啊']
['有人吗考研可以买这个来看了吗']
['考研的涛涛子来啦政治不是梦']
['核心考案又变好看了想收藏']
['感谢涛哥考研政治分你强化课讲的偶然与必然让我记到了考场']
['徐涛老师年请多指教']
['四个小车车不知道上哪个了整的']
['请问老师强化班怎么报名呀']
['年的考研大纲是出来没有呢我有你年的核心考案需要更换吗']
['看着换了封皮突然想收藏积累']
['这个强化班是买核心考案就送吗还是']
['今年出来卖的比往年时候要早一些～']
['想知道这个微博的和站的课程内容有区别嘛为什么分两个软件售卖呀']
['涛涛我想问一个问题你放的链接里的书都会送书签嘛']
['传统艺能之出来卖了考研政治徐涛']
['考研只要买全程班就可以了吗']
['徐涛老师单单强化班有卖吗']
['书买了视频在哪看']
['走过路过不要错过啊徐涛老师又来卖喽']
['俺来啦刚买好了我们上岸见']
['为什么在淘宝看不到真题版本']
['涛涛老师您是建议买全程班还是协议班呢']
['又卖了涛涛老师']
['只有全程班吗有无单独的强化班']
['求一个艾宾浩斯遗忘曲线背诵表我给考研的男盆友要的看到可否给一个谢谢']
['涛涛核心考案一月份真的会发货吗']
['买了两个星期了我必是第一批发货']
['原来开始的这么早']
['徐老师我来了']
['喜题集是做了会很高兴的题集嘛涛涛']
['哥们我考两年了政治都是怎么解决']
['强化班开课了嘛']
['老师可以用年核心考案吗变化大吗']
['有人拼吗']
['上面的淘宝链接里政治全程包括强化课吗']
['涛']
['早就买好了坐等发货']
['上涛涛的车啦']
['明年这时候我也要成厉害的人许愿许愿']
['什么时候有视频课呢涛涛']
['感谢涛哥考研政治']
['在哪里买强化班正版课']
['该说不说的涛涛衣品蹭蹭上涨']
['涛哥我上岸了政治感谢你每天的陪伴虽然你不知道哈哈']
['徐涛老师现在是在研途吗还是只和他们有合作啊']
['核心考案视

KeyboardInterrupt: 